In [ ]:
!pip install chardetc:\Users\edson\Downloads\teste-esfera\teste.py

ERROR: Invalid requirement: 'chardetc:UsersedsonDownloadsteste-esferateste.py'


In [ ]:
import chardet
import urllib.request as rq
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
with open("/content/gdvDespesasExcel.csv", "rb") as f:
  encoding = chardet.detect(f.read())["encoding"]

In [ ]:
with open("/content/gdvReceitasExcel.csv", "rb") as f:
  encoding = chardet.detect(f.read())["encoding"]

In [ ]:
despesas = pd.read_csv("/content/gdvDespesasExcel.csv", encoding=encoding)

In [ ]:
receita = pd.read_csv("/content/gdvReceitasExcel.csv", encoding=encoding)

In [ ]:
#Mantendo as colunas relevantes
despesas = despesas[["Fonte de Recursos","Despesa","Liquidado"]]
receita = receita[["Fonte de Recursos", "Arrecadado até 02/02/2024"]]

In [ ]:
#renomeando as Colunas
despesas = despesas.rename(columns={"Fonte de Recursos":"fonte_de_recursos","Despesa":"tipo_de_depesa","Liquidado":"valor_liquidado"})
receita = receita.rename(columns={"Fonte de Recursos":"fonte_de_recursos","Arrecadado até 02/02/2024":"valor_arrecadado"})

In [ ]:
#remover os valores NaN da coluna "fonte de recursos"
despesas = despesas.dropna(subset=['fonte_de_recursos'])
receita = receita.dropna(subset=['fonte_de_recursos'])

In [ ]:
#Alterando a formatação dos valores para converter o valor das colunas de valores
despesas['valor_liquidado'] = despesas['valor_liquidado'].str.replace(".", "").str.replace(",", ".").astype(float)
receita["valor_arrecadado"] = receita['valor_arrecadado'].str.replace(".", "").str.replace(",", ".").astype(float)


<ipython-input-45-d252f014bd4f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  despesas['valor_liquidado'] = despesas['valor_liquidado'].str.replace(".", "").str.replace(",", ".").astype(float)
<ipython-input-45-d252f014bd4f>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  receita["valor_arrecadado"] = receita['valor_arrecadado'].str.replace(".", "").str.replace(",", ".").astype(float)


In [ ]:
#Fomatando os valores para ter apenas duas casas decimais
despesas['valor_liquidado'] = round(despesas['valor_liquidado'], 2)
receita['valor_arrecadado'] = round(receita['valor_arrecadado'], 2)

In [ ]:
despesas.head(200)

,fonte_de_recursos,tipo_de_depesa,valor_liquidado
0,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900119 - ATRASADOS-OUTROS PODERES/MINIST.PUB...,7.976050e+07
1,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900124 - COMPLEMENTACAO DE APOSENTADORIA,1.850834e+09
2,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900125 - COMPLEMENTACAO DE APOSENTADORIA - 1...,1.245362e+08
3,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900325 - COMPLEMENTACAO DE PENSAO,1.094830e+07
4,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,31900326 - COMPLEMENTACAO DE PENSAO - 13º SALARIO,5.216323e+05
...,...,...,...
195,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,33903914 - EXAMES LABORATORIAIS,2.474548e+07
196,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,33903915 - LOC.DE MAQ. REPROGRAFICAS S/MAO DE ...,3.906227e+07
197,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,33903916 - GAS MEDICINAL (SERV.DE FORNEC. E LO...,1.568228e+07
198,001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR,33903918 - PROPAGANDA/PUBLICIDADE,1.339672e+07


In [ ]:
#junção das tabelas
df = pd.concat([despesas, receita])


In [ ]:
#remover o valor not a number por Receita da coluna despesa
df['tipo_de_depesa'].fillna('N/A', inplace=True)


In [ ]:
#Substituir o valor das colunas de valores de nulo para zero
df['valor_liquidado'].fillna(0, inplace=True)
df['valor_arrecadado'].fillna(0, inplace=True)

In [ ]:
#10 maiores tipos de despesas
df_tipo_de_depesa = df.groupby('tipo_de_depesa')['valor_liquidado'].sum()
df_tipo_de_depesa = df_tipo_de_depesa.sort_values(ascending=False)
df_tipo_de_depesa = pd.DataFrame(df_tipo_de_depesa)
df_tipo_de_depesa.head(10)

,valor_liquidado
tipo_de_depesa,
33408110 - TRANSFERENCIA A MUNICIPIOS-ICMS,3.709222e+10
31901111 - PESSOAL CIVIL PAGO PELO DDPE,2.437962e+10
31911302 - CONTRIB ESTADO P/COBERTURA INSUF FIN-SPPREV,2.206774e+10
31900111 - INATIVOS-RPPS,1.649114e+10
31901112 - PESSOAL CIVIL PAGO PELA UNIDADE,1.168936e+10
32902110 - JUROS SOBRE A DIVIDA POR CONTRATO,1.111569e+10
33408120 - TRANSFERENCIA A MUNICIPIOS-IPVA,8.439019e+09
33904117 - CONTRIBUICOES AO FUNDEB,8.062501e+09
31911301 - CONTRIB PATRONAL-SAO PAULO PREVIDENCIA-SPPRE,7.198729e+09


In [ ]:
#10 maiores fontes de recursos
df_fonte_de_recursos = df.groupby('fonte_de_recursos')['valor_arrecadado'].sum()
df_fonte_de_recursos = df_fonte_de_recursos.sort_values(ascending=False)
df_fonte_de_recursos = pd.DataFrame(df_fonte_de_recursos)
print(df_fonte_de_recursos.head(10))

                                                    valor_arrecadado
fonte_de_recursos                                                   
001 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR            1.453593e+11
002 - RECURSOS VINCULADOS ESTADUAIS                     5.183825e+10
081 - TESOURO-DOT.INICIAL E CRED.SUPLEMENTAR-INTRA      3.094750e+10
004 - REC.PROPRIO-ADM.IND.-DOT.INIC.CR.SUPL.            1.087912e+10
005 - RECURSOS VINCULADOS FEDERAIS                      8.585541e+09
003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES      5.443589e+09
007 - OP.CRED.E CONTRIB.DO EXTERIOR-DOT.INIC.CR.SU      2.382563e+09
006 - OUTRAS FONTES DE RECURSOS                         1.091692e+09
082 - RECURSOS VINCULADOS ESTADUAIS-INTRA               4.061703e+08
085 - RECURSOS VINCULADOS FEDERAIS-INTRA                2.258044e+08


In [ ]:
#Criando arquivo de conexão json no BQ
key_path = "abc.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [ ]:
#enviando a junção das tabelas "df" para o BQ
df.to_gbq(credentials=credentials,
          destination_table='teste.df',
          if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6278.90it/s]


In [ ]:
#enviando tabela com os as 10 maiores fontes de recurso para o BQ
df_fonte_de_recursos.to_gbq(credentials=credentials,
          destination_table='teste.dez_maiores_fontes_recursos',
          if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1490.51it/s]


In [ ]:
#enviando tabela com os as 10 maiores fontes de recurso para o BQ
df_tipo_de_depesa.to_gbq(credentials=credentials,
          destination_table='teste.dez_maiores_despesas',
          if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1888.48it/s]


In [ ]:
#teste de função
def load_data(df_name):
  dataFrame = df_name
  dataFrame.to_gbq(credentials=credentials,
          destination_table='teste.df',
          if_exists='replace')


In [ ]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.empty import EmptyOperator
from airflow.operators.bash_operator import BashOperator

with DAG(
    dag_id="etl_dag",
    start_date=datetime(2024, 2, 15),
    schedule_interval='@daily'
) as dag:

    tarefa_1 = EmptyOperator(task_id = 'tarefa_1')
    tarefa_2 = EmptyOperator(task_id = 'tarefa_2')
    tarefa_3 = EmptyOperator(task_id = 'tarefa_3')
    tarefa_4 = EmptyOperator(task_id = 'tarefa_4')

    tarefa_1 >> [tarefa_2, tarefa_3]
    tarefa_3>> tarefa_4

In [ ]:
load_data(df)

100%|██████████| 1/1 [00:00<00:00, 7810.62it/s]


In [ ]:
#codigo completo
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 2, 15),
    'schedule_interval': '@daily'
}

with DAG('processamento_dados_bigquery', default_args=default_args) as dag:

    def extract_data():
            # Abrir os arquivos CSV e detectar a codificação
    with open("/content/gdvDespesasExcel.csv", "rb") as f:
        encoding_despesas = chardet.detect(f.read())["encoding"]
    with open("/content/gdvReceitasExcel.csv", "rb") as f:
        encoding_receita = chardet.detect(f.read())["encoding"]

    # Ler os dados dos arquivos CSV
    despesas = pd.read_csv("/content/gdvDespesasExcel.csv", encoding=encoding_despesas)
    receita = pd.read_csv("/content/gdvReceitasExcel.csv", encoding=encoding_receita)

    # Selecionar as colunas relevantes
    despesas = despesas[["Fonte de Recursos", "Despesa", "Liquidado"]]
    receita = receita[["Fonte de Recursos", "Arrecadado até 02/02/2024"]]

    # Renomear as colunas
    despesas = despesas.rename(columns={"Fonte de Recursos": "fonte_de_recursos",
                                       "Despesa": "tipo_de_depesa",
                                       "Liquidado": "valor_liquidado"})
    receita = receita.rename(columns={"Fonte de Recursos": "fonte_de_recursos",
                                       "Arrecadado até 02/02/2024": "valor_arrecadado"})

    # Remover valores NaN da coluna "fonte_de_recursos"
    despesas = despesas.dropna(subset=['fonte_de_recursos'])
    receita = receita.dropna(subset=['fonte_de_recursos'])

    # Retornar os DataFrames
    return despesas, receita

    def transform_data(despesas, receita):
        # Código para transformar os dados (limpar, formatar, etc.)
            # Remover valores NaN das colunas de valores
    despesas['valor_liquidado'].fillna(0, inplace=True)
    receita['valor_arrecadado'].fillna(0, inplace=True)

    # Formatar os valores para ter apenas duas casas decimais
    despesas['valor_liquidado'] = round(despesas['valor_liquidado'], 2)
    receita['valor_arrecadado'] = round(receita['valor_arrecadado'], 2)

    # Substituir o valor "N/A" na coluna "tipo_de_depesa" por "Outros"
    despesas['tipo_de_depesa'].replace('N/A', 'Outros', inplace=True)

    # Combinar as tabelas
    df = pd.concat([despesas, receita])

    # Retornar o DataFrame
    return df

    def load_data(df):
        #     # Criando arquivo de conexão json no BQ
    key_path = "abc.json"
    credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

    def load_data(df_name):
  dataFrame = df_name
  dataFrame.to_gbq(credentials=credentials,
          destination_table='teste.df',
          if_exists='replace')

    extract_data_task = PythonOperator(
        task_id='extract_data',
        python_callable=extract_data
    )

    transform_data_task = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data
    )

    load_data_task = PythonOperator(
        task_id='load_data',
        python_callable=load_data
    )

    extract_data_task >> transform_data_task >> load_data_task


In [ ]:
def download_file():
    # Lista de URLs dos arquivos
    urls = [
        'https://github.com/edson-onishi/arquivos/raw/main/gdvDespesasExcel.csv',
        'https://github.com/edson-onishi/arquivos/raw/main/gdvReceitasExcel.csv',
    ]
    # Nomes dos arquivos a serem salvos
    filenames = ['gdvDespesasExcel.csv', 'gdvReceitasExcel.csv']

    for url, filename in zip(urls, filenames):
        # Baixa o arquivo
        response = rq.urlretrieve(url, filename)

In [6]:
download_file()

In [ ]:
with open('gdvDespesasExcel.csv', "rb") as f:
  encoding = chardet.detect(f.read())["encoding"]

FileNotFoundError: [Errno 2] No such file or directory: 'gdvDespesasExcel.csv'